In [10]:
"""
Multimodal Evaluation Metrics for LSTMABAR
Implements STS, Spectral Centroid Error, and MFCC Similarity
"""

import torch
import numpy as np
import librosa
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import ttest_rel
import pandas as pd
from typing import Dict, List, Tuple, Optional
from pathlib import Path

from lstmabar_model import LSTMABAR

In [7]:
class LSTMABAREvaluator:
    """
    Complete evaluation framework for LSTMABAR model
    Implements text-side and audio-side metrics
    """
    
    def __init__(
        self,
        model: LSTMABAR,
        text_model_name: str = 'sentence-transformers/all-MiniLM-L6-v2',
        sample_rate: int = 44100
    ):
        self.model = model
        self.sample_rate = sample_rate
        
        # Text encoder for STS
        self.text_encoder = SentenceTransformer(text_model_name)
        
        # Archetype descriptors for mapping
        self.archetype_descriptors = {
            'sine': 'smooth pure warm mellow soft gentle flowing',
            'square': 'digital harsh buzzy retro electronic robotic',
            'sawtooth': 'bright cutting metallic sharp aggressive',
            'triangle': 'hollow woody muted filtered organic',
            'noise': 'rough textured grainy distorted chaotic'
        }
        
        # Expected spectral centroids for common descriptors (Hz)
        self.centroid_map = {
            'bright': 3500, 'dark': 800, 'warm': 1000, 'harsh': 4000,
            'mellow': 1200, 'sharp': 5000, 'smooth': 1500, 'soft': 1100,
            'aggressive': 3800, 'gentle': 1300, 'cutting': 4200
        }
    
    # ==================== TEXT-SIDE EVALUATION (STS) ====================
    
    def compute_sts(
        self,
        input_description: str,
        predicted_archetype_weights: np.ndarray
    ) -> float:
        """
        Semantic Textual Similarity: Does model understand description?
        
        Args:
            input_description: Original user description
            predicted_archetype_weights: Model's predicted archetype weights (5,)
        
        Returns:
            STS score (0-1, higher is better)
        """
        # Convert archetype weights to descriptive text
        predicted_description = self._archetypes_to_text(predicted_archetype_weights)
        
        # Encode both descriptions
        emb_input = self.text_encoder.encode(input_description, convert_to_tensor=True)
        emb_predicted = self.text_encoder.encode(predicted_description, convert_to_tensor=True)
        
        # Compute cosine similarity
        sts_score = util.cos_sim(emb_input, emb_predicted).item()
        
        return sts_score
    
    def _archetypes_to_text(
        self,
        archetype_weights: np.ndarray,
        threshold: float = 0.1
    ) -> str:
        """Convert archetype mixture to natural language"""
        archetype_names = ['sine', 'square', 'sawtooth', 'triangle', 'noise']
        
        # Get top contributing archetypes
        descriptions = []
        for i, (name, weight) in enumerate(zip(archetype_names, archetype_weights)):
            if weight > threshold:
                descriptions.append(self.archetype_descriptors[name])
        
        return ' '.join(descriptions) if descriptions else 'neutral sound'
    
    # ==================== AUDIO-SIDE EVALUATION ====================
    
    def spectral_centroid_error(
        self,
        output_audio: np.ndarray,
        target_description: str
    ) -> float:
        """
        Measures brightness accuracy (Hz error)
        
        Args:
            output_audio: Generated audio array
            target_description: Target description
        
        Returns:
            Absolute error in Hz (lower is better)
        """
        # Compute actual spectral centroid
        centroid = librosa.feature.spectral_centroid(
            y=output_audio,
            sr=self.sample_rate
        )
        actual_centroid = np.mean(centroid)
        
        # Determine expected centroid from description
        expected_centroid = self._description_to_centroid(target_description)
        
        # Calculate error
        error_hz = abs(actual_centroid - expected_centroid)
        
        return error_hz
    
    def _description_to_centroid(self, description: str) -> float:
        """Map description to expected spectral centroid"""
        words = description.lower().split()
        expected_centroids = []
        
        for word in words:
            if word in self.centroid_map:
                expected_centroids.append(self.centroid_map[word])
        
        if expected_centroids:
            return np.mean(expected_centroids)
        
        return 2000  # Neutral default
    
    def mfcc_similarity(
        self,
        output_audio: np.ndarray,
        reference_audio: np.ndarray,
        n_mfcc: int = 13
    ) -> float:
        """
        Measures timbral similarity (0-1, higher is better)
        
        Args:
            output_audio: Generated audio array
            reference_audio: Target/reference audio array
            n_mfcc: Number of MFCC coefficients
        
        Returns:
            Cosine similarity (1 is perfect match, >0.8 is good)
        """
        # Extract MFCCs from both signals
        mfcc_output = librosa.feature.mfcc(
            y=output_audio,
            sr=self.sample_rate,
            n_mfcc=n_mfcc
        )
        mfcc_reference = librosa.feature.mfcc(
            y=reference_audio,
            sr=self.sample_rate,
            n_mfcc=n_mfcc
        )
        
        # Average across time
        mfcc_output_mean = np.mean(mfcc_output, axis=1).reshape(1, -1)
        mfcc_reference_mean = np.mean(mfcc_reference, axis=1).reshape(1, -1)
        
        # Compute cosine similarity
        similarity = cosine_similarity(mfcc_output_mean, mfcc_reference_mean)[0][0]
        
        return similarity
    
    # ==================== COMPREHENSIVE EVALUATION ====================
    
    def evaluate_single_transformation(
        self,
        input_audio: np.ndarray,
        input_description: str,
        output_audio: np.ndarray,
        reference_audio: np.ndarray,
        predicted_weights: np.ndarray
    ) -> Dict[str, float]:
        """
        Complete evaluation of a single transformation
        
        Returns:
            Dict with all metrics
        """
        results = {
            # Text-side (NLP understanding)
            'sts_score': self.compute_sts(input_description, predicted_weights),
            
            # Audio-side (transformation quality)
            'spectral_centroid_error_hz': self.spectral_centroid_error(
                output_audio, input_description
            ),
            'mfcc_similarity': self.mfcc_similarity(output_audio, reference_audio)
        }
        
        return results
    
    def evaluate_model(
        self,
        test_samples: List[Dict],
        generate_reference: bool = True
    ) -> pd.DataFrame:
        """
        Evaluate model on test set
        
        Args:
            test_samples: List of dicts with 'audio', 'description', 'target_weights'
            generate_reference: If True, generate reference audio from target weights
        
        Returns:
            DataFrame with results for each sample
        """
        self.model.eval()
        results = []
        
        print(f"Evaluating model on {len(test_samples)} samples...")
        
        for i, sample in enumerate(test_samples):
            input_audio = sample['audio']
            description = sample['description']
            target_weights = sample['target_weights']
            
            # Convert to tensor
            audio_tensor = torch.from_numpy(input_audio).unsqueeze(0).float().to(self.model.device)
            
            # Model inference
            with torch.no_grad():
                transformed, metadata = self.model.inference([description], audio_tensor)
            
            # Convert back to numpy
            output_audio = transformed[0].cpu().numpy()
            predicted_weights = metadata['predicted_weights'][0]
            
            # Generate reference audio if needed
            if generate_reference:
                # Use archetype generator to create ideal sound
                reference_audio = self._generate_reference_audio(target_weights)
            else:
                reference_audio = input_audio  # Fallback
            
            # Evaluate
            metrics = self.evaluate_single_transformation(
                input_audio,
                description,
                output_audio,
                reference_audio,
                predicted_weights
            )
            
            # Add metadata
            metrics['description'] = description
            metrics['sample_idx'] = i
            
            results.append(metrics)
            
            if (i + 1) % 10 == 0:
                print(f"  Processed {i+1}/{len(test_samples)} samples")
        
        # Convert to DataFrame
        df = pd.DataFrame(results)
        
        return df
    
    def _generate_reference_audio(
        self,
        archetype_weights: np.ndarray,
        duration: float = 2.0
    ) -> np.ndarray:
        """Generate reference audio from archetype mixture"""
        n_samples = int(self.sample_rate * duration)
        t = np.linspace(0, duration, n_samples, endpoint=False)
        frequency = 440  # A4
        
        audio = np.zeros(n_samples)
        
        # Generate each archetype component
        archetypes = {
            'sine': np.sin(2 * np.pi * frequency * t),
            'square': np.sign(np.sin(2 * np.pi * frequency * t)),
            'sawtooth': 2 * (t * frequency - np.floor(0.5 + t * frequency)),
            'triangle': 2 * np.abs(2 * (t * frequency - np.floor(t * frequency + 0.5))) - 1,
            'noise': np.random.randn(n_samples) * 0.3
        }
        
        archetype_names = ['sine', 'square', 'sawtooth', 'triangle', 'noise']
        
        # Mix according to weights
        for i, name in enumerate(archetype_names):
            audio += archetype_weights[i] * archetypes[name]
        
        # Normalize
        audio = audio / (np.max(np.abs(audio)) + 1e-8) * 0.95
        
        return audio
    
    def compare_with_baseline(
        self,
        test_samples: List[Dict],
        baseline_model: Optional[object] = None
    ) -> pd.DataFrame:
        """
        Compare LSTMABAR with baseline model
        
        Args:
            test_samples: Test samples
            baseline_model: Baseline model (if None, uses simple keyword matching)
        
        Returns:
            Comparison DataFrame
        """
        # Evaluate LSTMABAR
        lstmabar_results = self.evaluate_model(test_samples)
        
        # Evaluate baseline
        if baseline_model is None:
            baseline_results = self._evaluate_keyword_baseline(test_samples)
        else:
            baseline_results = baseline_model.evaluate(test_samples)
        
        # Compute statistics
        comparison = self._compute_comparison_stats(lstmabar_results, baseline_results)
        
        return comparison
    
    def _evaluate_keyword_baseline(self, test_samples: List[Dict]) -> pd.DataFrame:
        """Evaluate simple keyword-matching baseline"""
        results = []
        
        # Simple keyword → archetype mapping
        keyword_map = {
            'bright': [0.1, 0.1, 0.6, 0.1, 0.1],  # Mostly sawtooth
            'warm': [0.6, 0.1, 0.1, 0.2, 0.0],    # Mostly sine
            'harsh': [0.1, 0.5, 0.2, 0.1, 0.1],   # Mostly square
            'smooth': [0.6, 0.1, 0.1, 0.2, 0.0],  # Mostly sine
            'distorted': [0.1, 0.2, 0.2, 0.1, 0.4] # High noise
        }
        
        for i, sample in enumerate(test_samples):
            description = sample['description'].lower()
            
            # Find matching keywords
            predicted_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # Uniform default
            for keyword, weights in keyword_map.items():
                if keyword in description:
                    predicted_weights = np.array(weights)
                    break
            
            # Simple transformation (just apply gain based on brightness)
            output_audio = sample['audio'].copy()
            if 'bright' in description:
                # Boost high frequencies (simplified)
                output_audio = output_audio * 1.2
            
            reference_audio = self._generate_reference_audio(sample['target_weights'])
            
            # Evaluate
            metrics = self.evaluate_single_transformation(
                sample['audio'],
                description,
                output_audio,
                reference_audio,
                predicted_weights
            )
            
            metrics['description'] = description
            metrics['sample_idx'] = i
            results.append(metrics)
        
        return pd.DataFrame(results)
    
    def _compute_comparison_stats(
        self,
        lstmabar_results: pd.DataFrame,
        baseline_results: pd.DataFrame
    ) -> pd.DataFrame:
        """Compute statistical comparison"""
        metrics = ['sts_score', 'spectral_centroid_error_hz', 'mfcc_similarity']
        
        comparison = {
            'Metric': [],
            'Baseline_Mean': [],
            'Baseline_Std': [],
            'LSTMABAR_Mean': [],
            'LSTMABAR_Std': [],
            'Improvement': [],
            'p_value': []
        }
        
        for metric in metrics:
            baseline_vals = baseline_results[metric].values
            lstmabar_vals = lstmabar_results[metric].values
            
            baseline_mean = np.mean(baseline_vals)
            baseline_std = np.std(baseline_vals)
            lstmabar_mean = np.mean(lstmabar_vals)
            lstmabar_std = np.std(lstmabar_vals)
            
            comparison['Metric'].append(metric)
            comparison['Baseline_Mean'].append(f"{baseline_mean:.4f}")
            comparison['Baseline_Std'].append(f"{baseline_std:.4f}")
            comparison['LSTMABAR_Mean'].append(f"{lstmabar_mean:.4f}")
            comparison['LSTMABAR_Std'].append(f"{lstmabar_std:.4f}")
            
            # Calculate improvement
            if 'error' in metric.lower():
                # Lower is better
                improvement = ((baseline_mean - lstmabar_mean) / baseline_mean) * 100
            else:
                # Higher is better
                improvement = ((lstmabar_mean - baseline_mean) / baseline_mean) * 100
            
            comparison['Improvement'].append(f"{improvement:.2f}%")
            
            # Paired t-test
            _, p_val = ttest_rel(baseline_vals, lstmabar_vals)
            comparison['p_value'].append(f"{p_val:.4f}")
        
        return pd.DataFrame(comparison)
    
    def generate_evaluation_report(
        self,
        test_samples: List[Dict],
        save_path: Optional[str] = None
    ):
        """Generate comprehensive evaluation report"""
        print("\n" + "="*80)
        print("LSTMABAR MODEL EVALUATION REPORT")
        print("="*80)
        
        # Evaluate model
        results = self.evaluate_model(test_samples)
        
        # Compare with baseline
        comparison = self.compare_with_baseline(test_samples)
        
        print("\n" + "="*80)
        print("COMPARISON WITH BASELINE")
        print("="*80)
        print(comparison.to_string(index=False))
        
        print("\n" + "="*80)
        print("INTERPRETATION")
        print("="*80)
        
        # STS interpretation
        sts_mean = results['sts_score'].mean()
        print(f"\n📝 TEXT UNDERSTANDING (STS Score):")
        print(f"   Mean: {sts_mean:.3f}")
        if sts_mean > 0.75:
            print(f"   ✓ Excellent semantic understanding!")
        elif sts_mean > 0.60:
            print(f"   → Good semantic understanding")
        else:
            print(f"   ⚠ Needs improvement in NLP comprehension")
        
        # Spectral centroid interpretation
        sc_mean = results['spectral_centroid_error_hz'].mean()
        print(f"\n🎵 BRIGHTNESS ACCURACY (Spectral Centroid Error):")
        print(f"   Mean: {sc_mean:.0f} Hz")
        if sc_mean < 200:
            print(f"   ✓ Excellent brightness targeting!")
        elif sc_mean < 500:
            print(f"   → Good brightness control")
        else:
            print(f"   ⚠ Brightness targeting needs improvement")
        
        # MFCC interpretation
        mfcc_mean = results['mfcc_similarity'].mean()
        print(f"\n🎸 TIMBRE QUALITY (MFCC Similarity):")
        print(f"   Mean: {mfcc_mean:.3f}")
        if mfcc_mean > 0.80:
            print(f"   ✓ Excellent timbral matching!")
        elif mfcc_mean > 0.65:
            print(f"   → Good timbral similarity")
        else:
            print(f"   ⚠ Timbre quality needs improvement")
        
        print("\n" + "="*80)
        
        # Save detailed results
        if save_path:
            results.to_csv(save_path, index=False)
            comparison.to_csv(save_path.replace('.csv', '_comparison.csv'), index=False)
            print(f"\nDetailed results saved to {save_path}")

    def evaluate_model_on_test_set(
        self,
        test_data_path: str,
        max_samples: Optional[int] = None,
        save_results: bool = True,
        results_path: str = 'test_results.csv'
    ) -> pd.DataFrame:
        """
        Evaluate model on test set from MusicCaps
        
        Args:
            test_data_path: Path to test .npz file
            max_samples: Max samples to evaluate (None = all)
            save_results: Whether to save results
            results_path: Path to save results
        
        Returns:
            DataFrame with evaluation results
        """
        print(f"\n{'='*80}")
        print("EVALUATING ON TEST SET")
        print(f"{'='*80}\n")
        
        # Load test data
        print(f"Loading test data from {test_data_path}...")
        data = np.load(test_data_path, allow_pickle=True)
        
        vectors = data['archetype_vectors']
        descriptions = data['descriptions'].tolist()
        audio_paths = data['audio_paths'].tolist()
        
        n_samples = min(len(descriptions), max_samples) if max_samples else len(descriptions)
        print(f"Evaluating on {n_samples} test samples\n")
        
        # Prepare test samples
        test_samples = []
        for i in range(n_samples):
            audio_path = audio_paths[i]
            
            # Check if audio exists
            if not Path(audio_path).exists():
                print(f"Skipping {i}: audio file not found")
                continue
            
            # Load audio
            try:
                audio, sr = librosa.load(audio_path, sr=self.sample_rate, duration=2.0)
                
                # Pad or trim
                target_length = int(self.sample_rate * 2.0)
                if len(audio) < target_length:
                    audio = np.pad(audio, (0, target_length - len(audio)))
                else:
                    audio = audio[:target_length]
                
                test_samples.append({
                    'audio': audio,
                    'description': descriptions[i],
                    'target_weights': vectors[i]
                })
            except Exception as e:
                print(f"Error loading {audio_path}: {e}")
                continue
        
        print(f"Successfully loaded {len(test_samples)} test samples\n")
        
        # Evaluate
        results_df = self.evaluate_model(test_samples, generate_reference=True)
        
        # Save results
        if save_results:
            results_df.to_csv(results_path, index=False)
            print(f"\n✓ Results saved to {results_path}")
        
        return results_df

In [3]:
from lstmabar_model import LSTMABAR

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load trained model
print("Loading LSTMABAR model...")
model = LSTMABAR(
    embedding_dim=768,
    audio_architecture='resnet',
    sample_rate=44100,
    device=device
)

Loading LSTMABAR model...
Loading text encoder: sentence-transformers/all-MiniLM-L6-v2


In [14]:
# Load checkpoint
model.load_checkpoint('checkpoints/best_model.pth')

# Initialize evaluator
evaluator = LSTMABAREvaluator(model, sample_rate=44100)

Model loaded from checkpoints/best_model.pth (epoch 1)


In [5]:
# Create test samples
print("\nCreating test samples...")
test_samples = []

# Example 1: Bright guitar
audio1 = np.random.randn(44100 * 2)  # 2 seconds
test_samples.append({
    'audio': audio1,
    'description': 'bright and cutting guitar with metallic tone',
    'target_weights': np.array([0.1, 0.1, 0.6, 0.1, 0.1])
})

# Example 2: Warm piano
audio2 = np.random.randn(44100 * 2)
test_samples.append({
    'audio': audio2,
    'description': 'warm smooth piano melody with gentle sustain',
    'target_weights': np.array([0.6, 0.05, 0.1, 0.2, 0.05])
})

# Example 3: Digital synth
audio3 = np.random.randn(44100 * 2)
test_samples.append({
    'audio': audio3,
    'description': 'harsh digital synth with buzzy retro sound',
    'target_weights': np.array([0.1, 0.55, 0.15, 0.1, 0.1])
})


Creating test samples...


In [6]:
# Run evaluation
print("\nRunning evaluation...")
evaluator.generate_evaluation_report(
    test_samples,
    save_path='evaluation_results.csv'
)

print("\n✓ Evaluation complete!")


Running evaluation...

LSTMABAR MODEL EVALUATION REPORT
Evaluating model on 3 samples...
Evaluating model on 3 samples...

COMPARISON WITH BASELINE
                    Metric Baseline_Mean Baseline_Std LSTMABAR_Mean LSTMABAR_Std Improvement p_value
                 sts_score        0.5507       0.1440        0.4061       0.1241     -26.26%  0.3797
spectral_centroid_error_hz     7992.9986    1249.6807     7252.6950    1247.1636       9.26%  0.0000
           mfcc_similarity       -0.6586       0.1760       -0.6140       0.1816      -6.77%  0.0497

INTERPRETATION

📝 TEXT UNDERSTANDING (STS Score):
   Mean: 0.406
   ⚠ Needs improvement in NLP comprehension

🎵 BRIGHTNESS ACCURACY (Spectral Centroid Error):
   Mean: 7253 Hz
   ⚠ Brightness targeting needs improvement

🎸 TIMBRE QUALITY (MFCC Similarity):
   Mean: -0.613
   ⚠ Timbre quality needs improvement


Detailed results saved to evaluation_results.csv

✓ Evaluation complete!


In [15]:
test_path = 'musiccaps_training_data_test.npz'

test_results = evaluator.evaluate_model_on_test_set(
    test_data_path=test_path,
    max_samples=100,
    save_results=True,
    results_path='test_results.csv'
)

print(f"\nTest Performance:")
print(f"  STS: {test_results['sts_score'].mean():.3f}")
print(f"  SC Error: {test_results['spectral_centroid_error_hz'].mean():.0f} Hz")
print(f"  MFCC: {test_results['mfcc_similarity'].mean():.3f}")


EVALUATING ON TEST SET

Loading test data from musiccaps_training_data_test.npz...
Evaluating on 73 test samples

Successfully loaded 73 test samples

Evaluating model on 73 samples...
  Processed 10/73 samples
  Processed 20/73 samples
  Processed 30/73 samples
  Processed 40/73 samples
  Processed 50/73 samples
  Processed 60/73 samples
  Processed 70/73 samples

✓ Results saved to test_results.csv

Test Performance:
  STS: 0.214
  SC Error: 5533 Hz
  MFCC: 0.384
